# Example of using PxWebApi version 2 in Python


## The new API offers both HTTP POST as before and HTTP GET which is new


#### uses the pyjstat library for JSON-stat


In [1]:
import pandas as pd
import requests
from pyjstat import pyjstat 
import io 

## POST query and get Pandas dataframe in return
New: Language and format are specified in the URL


In [2]:
# Salmon export
POST_URL = 'https://data.ssb.no/api/pxwebapi/v2/tables/03024/data?lang=en&outputFormat=json-stat2'

API query - Different structure than in PxWebApi Version 1 and not backwards compatible,

- fetches frozen and fresh salmon from week1 2025 using the new `from(2025U01)`


In [3]:

payload = {"selection": [
			{"variableCode": "VareGrupper2", 
             "valueCodes": [
                 "01", "02"
             ] 
            },
			{"variableCode": "ContentsCode", 
             "valueCodes": [
                 "Vekt", "Kilopris"
             ] 
            },
			{"variableCode": "Tid", 
             "valueCodes": ["from(2025U01)"] }  # V2 
		]
		}



In [4]:
result = requests.post(POST_URL, json = payload)

In [5]:
print(result)

<Response [200]>


Result only gives HTTP status code - 200 if OK. Body is in resultat.text


In [6]:
dataset = pyjstat.Dataset.read(result.text)

In [7]:
df = dataset.write('dataframe')

In [8]:
df.head()

,commodity group,contents,week,value
0,"Fish-farm bred salmon, fresh or chilled",Weight (tonnes),2025U01,11364.0
1,"Fish-farm bred salmon, fresh or chilled",Weight (tonnes),2025U02,15939.0
2,"Fish-farm bred salmon, fresh or chilled",Weight (tonnes),2025U03,17325.0
3,"Fish-farm bred salmon, fresh or chilled",Weight (tonnes),2025U04,15564.0
4,"Fish-farm bred salmon, fresh or chilled",Weight (tonnes),2025U05,16370.0


In [9]:
df.tail()

,commodity group,contents,week,value
167,"Fish-farm bred salmon, frozen",Price per kilo (NOK),2025U39,71.33
168,"Fish-farm bred salmon, frozen",Price per kilo (NOK),2025U40,69.75
169,"Fish-farm bred salmon, frozen",Price per kilo (NOK),2025U41,72.00
170,"Fish-farm bred salmon, frozen",Price per kilo (NOK),2025U42,72.47
171,"Fish-farm bred salmon, frozen",Price per kilo (NOK),2025U43,74.86


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   commodity group  172 non-null    object 
 1   contents         172 non-null    object 
 2   week             172 non-null    object 
 3   value            172 non-null    float64
dtypes: float64(1), object(3)
memory usage: 5.5+ KB


In [11]:
print(result.text)

{"version":"2.0","class":"dataset","label":"03024: Export of salmon, fish-farm bred, by commodity group, contents and week","source":"Statistics Norway","updated":"2025-10-29T07:00:00Z","role":{"time":["Tid"],"metric":["ContentsCode"]},"id":["VareGrupper2","ContentsCode","Tid"],"size":[2,2,43],"dimension":{"VareGrupper2":{"label":"commodity group","category":{"index":{"01":0,"02":1},"label":{"01":"Fish-farm bred salmon, fresh or chilled","02":"Fish-farm bred salmon, frozen"}},"extension":{"elimination":false,"show":"value"}},"ContentsCode":{"label":"contents","category":{"index":{"Vekt":0,"Kilopris":1},"label":{"Vekt":"Weight (tonnes)","Kilopris":"Price per kilo (NOK)"},"unit":{"Vekt":{"base":"tonnes","decimals":0},"Kilopris":{"base":"NOK","decimals":2}}},"extension":{"elimination":false,"refperiod":{"Vekt":"End of week","Kilopris":"End of week"},"show":"value","measuringType":{"Vekt":"Flow","Kilopris":"Flow"},"priceType":{"Vekt":"NotApplicable","Kilopris":"NotApplicable"},"adjustment"

## Retrieves some metadata from the JSON-stat dataset

In [12]:
title = dataset['label']
print(title)

03024: Export of salmon, fish-farm bred, by commodity group, contents and week


In [13]:
last_update = dataset['updated']
print(last_update)

2025-10-29T07:00:00Z


In [14]:
last_update = dataset['updated']
print(last_update)

2025-10-29T07:00:00Z


In [15]:
source = dataset['source']
print(source)

Statistics Norway


'Role' gir snarveier til statistikkvariabel(metric), Tid og evt. Geo

In [16]:
ds_roles = dataset['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'metric': ['ContentsCode']})


Here are some new metadata that came in 2025. Applies both to V1 and V2 of the API

In [17]:
ds_id = dataset['extension']['px']['tableid']
print(ds_id)

03024


In [18]:
ds_shorttit = dataset['extension']['px']['contents']
print(ds_shorttit)

03024: Export of salmon, fish-farm bred,


Check if the table has got a footnote

In [19]:
if 'note' in dataset:
    ds_note = dataset['note']
    print(ds_note)

In [20]:
ds_dims = dataset['id']
ds_dims

['VareGrupper2', 'ContentsCode', 'Tid']

In [21]:
ds_dim = dataset['dimension']['VareGrupper2']['label']
ds_dim

'commodity group'

In [22]:
ds_shorttit,' etter ',ds_dim

('03024: Export of salmon, fish-farm bred,', ' etter ', 'commodity group')

In [23]:
ds_dimcat = dataset['dimension']['ContentsCode']
ds_dimcat

OrderedDict([('label', 'contents'),
             ('category',
              OrderedDict([('index',
                            OrderedDict([('Vekt', 0), ('Kilopris', 1)])),
                           ('label',
                            OrderedDict([('Vekt', 'Weight (tonnes)'),
                                         ('Kilopris',
                                          'Price per kilo (NOK)')])),
                           ('unit',
                            OrderedDict([('Vekt',
                                          OrderedDict([('base', 'tonnes'),
                                                       ('decimals', 0)])),
                                         ('Kilopris',
                                          OrderedDict([('base', 'NOK'),
                                                       ('decimals',
                                                        2)]))]))])),
             ('extension',
              OrderedDict([('elimination', False),
                   

In [24]:
ds_var = dataset['dimension']['VareGrupper2']['category']['label']['01']
ds_var

'Fish-farm bred salmon, fresh or chilled'

## GET - PxWebApi2

Pre-defined datasets are new in version 2 of PxWebApi. 

We use the POST URL above with http GET to give us a pre-defined dataset.

The parameter `?lang=no&outputFormat=json-stat2` is optional because it is set as default in `/config`

In [25]:
print(requests.get(POST_URL).url)

https://data.ssb.no/api/pxwebapi/v2/tables/03024/data?lang=en&outputFormat=json-stat2


In [26]:
resultat_get = requests.get(POST_URL)

In [27]:
res_get = resultat_get.text

In [28]:
dataset_get = pyjstat.Dataset.read(res_get)

In [29]:
# dataset_get

In [30]:
df_get = dataset_get.write('dataframe')

In [31]:
df_get

,commodity group,week,contents,value
0,"Fish-farm bred salmon, fresh or chilled",2025U43,Weight (tonnes),25886.00
1,"Fish-farm bred salmon, fresh or chilled",2025U43,Price per kilo (NOK),78.25
2,"Fish-farm bred salmon, frozen",2025U43,Weight (tonnes),740.00
3,"Fish-farm bred salmon, frozen",2025U43,Price per kilo (NOK),74.86


We can also use get-url with parameters: fresh salmon, weight, price in 2025

`&valueCodes[VareGrupper2]=01&valueCodes[ContentsCode]=Vekt,Kilopris&valueCodes[Tid]=from(2025U01)`

In [32]:
GET_URL = 'https://data.ssb.no/api/pxwebapi/v2/tables/03024/data?lang=en&valueCodes[VareGrupper2]=01&valueCodes[ContentsCode]=Vekt,Kilopris&valueCodes[Tid]=from(2025U01)'

In [33]:
resultat_get1 = requests.get(GET_URL)

In [34]:
dataset1_get = pyjstat.Dataset.read(resultat_get1.text)

In [35]:
df1 = dataset1_get.write('dataframe')

In [36]:
df1.tail(2)

,commodity group,contents,week,value
84,"Fish-farm bred salmon, fresh or chilled",Price per kilo (NOK),2025U42,73.74
85,"Fish-farm bred salmon, fresh or chilled",Price per kilo (NOK),2025U43,78.25


__New in API2.__ 

Parameters for controlling CSV output.

For the output formats csv, html and xlsx, you can specify the display of code/text and table title in `outputformatparams`:

- `UseCodes` (display codes) 
- `UseTexts` (display text) 
- `UseCodesAndTexts` (display both codes and text) 
- `IncludeTitle` (include table title) 

You can also use `stub` to specify which variables you want to place in the front column of the table and possibly `heading` for the variables you want to place in the table header. If you place all variables in stub, you get a so-called pivot-friendly table.


And for csv files you can choose between different separators:

- `SeparatorTab` (tab between columns)
- `SeparatorSpace` (space between columns)
- `SeparatorSemicolon` (semicolon between columns)

Mask one or more single character using `?`

new filters: `from(), [range(x, y)], bottom() ` 

The URL below the same as the POST query, but the output format is CSV. 

The output here is semicolon separated, with both code and text. Contentscode is in head, and the order of columns are Varegruppe2 og Tid.

`outputformat = csv`

`outputformatparams = separatorsemicolon, usecodesandtexts`

`heading = ContentsCode`

`stub = VareGrupper2, Tid`

In [37]:
CSV_URL = 'https://data.ssb.no/api/pxwebapi/v2/tables/03024/data?lang=en&outputformat=csv&outputformatparams=separatorsemicolon,usecodesandtexts&heading=ContentsCode&stub=VareGrupper2,Tid&valuecodes[ContentsCode]=Vekt,Kilopris&valuecodes[Varegrupper2]=01,02&valuecodes[Tid]=2025*'

It is possible to pass the URL with CSV directly to Pandas, with pd.read_csv(), and get a dataframe.

In [38]:
df = pd.read_csv(CSV_URL, delimiter = ';')

In [39]:
df.head(8)

,VareGrupper2 - commodity group,Tid - week,Vekt - Weight (tonnes),Kilopris - Price per kilo (NOK)
0,"01 - Fish-farm bred salmon, fresh or chilled",2025U01 - 2025U01,11364,125.36
1,"01 - Fish-farm bred salmon, fresh or chilled",2025U02 - 2025U02,15939,117.77
2,"01 - Fish-farm bred salmon, fresh or chilled",2025U03 - 2025U03,17325,105.08
3,"01 - Fish-farm bred salmon, fresh or chilled",2025U04 - 2025U04,15564,98.43
4,"01 - Fish-farm bred salmon, fresh or chilled",2025U05 - 2025U05,16370,94.49
5,"01 - Fish-farm bred salmon, fresh or chilled",2025U06 - 2025U06,15184,92.18
6,"01 - Fish-farm bred salmon, fresh or chilled",2025U07 - 2025U07,15966,93.23
7,"01 - Fish-farm bred salmon, fresh or chilled",2025U08 - 2025U08,15536,92.97


Same with requests. It's more cumbersome if you don't use the URL directly.

In [40]:
result_csv = requests.get(CSV_URL)

In [41]:
result_csv

<Response [200]>

In [42]:
r_csv = result_csv.text

In [43]:
r_csv

'"VareGrupper2 - commodity group";"Tid - week";"Vekt - Weight (tonnes)";"Kilopris - Price per kilo (NOK)"\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U01 - 2025U01";11364;125.36\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U02 - 2025U02";15939;117.77\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U03 - 2025U03";17325;105.08\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U04 - 2025U04";15564;98.43\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U05 - 2025U05";16370;94.49\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U06 - 2025U06";15184;92.18\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U07 - 2025U07";15966;93.23\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U08 - 2025U08";15536;92.97\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U09 - 2025U09";18067;92.67\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U10 - 2025U10";17116;87.00\r\n"01 - Fish-farm bred salmon, fresh or chilled";"2025U11 - 2025U11";1

In [44]:
df1 = pd.read_csv(io.StringIO(r_csv), delimiter = ';')

In [45]:
df1.tail(8)

,VareGrupper2 - commodity group,Tid - week,Vekt - Weight (tonnes),Kilopris - Price per kilo (NOK)
78,"02 - Fish-farm bred salmon, frozen",2025U36 - 2025U36,921,70.89
79,"02 - Fish-farm bred salmon, frozen",2025U37 - 2025U37,708,64.45
80,"02 - Fish-farm bred salmon, frozen",2025U38 - 2025U38,864,73.80
81,"02 - Fish-farm bred salmon, frozen",2025U39 - 2025U39,796,71.33
82,"02 - Fish-farm bred salmon, frozen",2025U40 - 2025U40,701,69.75
83,"02 - Fish-farm bred salmon, frozen",2025U41 - 2025U41,606,72.00
84,"02 - Fish-farm bred salmon, frozen",2025U42 - 2025U42,568,72.47
85,"02 - Fish-farm bred salmon, frozen",2025U43 - 2025U43,740,74.86
